In [1]:
import pandas as pd
import numpy as np

import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_manipulation import transform_data_datetime

# ERA5 datasets

Scope of the notebook: once downloaded datasets from Copernicus website, we implement functions to:
- Read `grib` files
- Produce Hourly, daily, monthly, and yearly dataframes for Total Precipitation and 2 meters Temperature

## Grib Files Parameters for London and Puglia region

Data are downloaded from the [Copernicus Website](https://cds.climate.copernicus.eu/requests?tab=all)

### London GRIB grid Parameters

For London, UK, you'll want to define a geographical area that encompasses the Greater London region. Here are suitable values for the grid boundaries:
- North: 51.7°N
- South: 51.3°N
- West: -0.5°W
- East: 0.3°E
These coordinates create a rectangular grid that covers Greater London and a small buffer around it. London's city center is approximately at 51.5°N, 0.1°W, so this grid provides good coverage while keeping the data volume manageable.

If you want a smaller area focused just on central London, you could use tighter bounds:

North: 51.6°N
South: 51.4°N
West: -0.3°W
East: 0.1°E

Or if you need a larger area covering the broader London metropolitan region, you could expand to:

North: 52.0°N
South: 51.0°N
West: -1.0°W
East: 1.0°E

The first set of coordinates (51.7°N, 51.3°N, -0.5°W, 0.3°E) is probably your best bet for most applications - it captures London proper without being unnecessarily large.


```
North: 51.6
West:  -0.2
South: 51.4
East:   0.1
```

### Puglia Region GRIB grid Parameters

For comparing London and Puglia climate data, here's a suitable approach:

- Grid Selection Strategy
Option 1: Two separate queries (Recommended)

Query London area: North 51.7°N, South 51.3°N, West -0.5°W, East 0.3°E
Query Puglia area: North 41.9°N, South 39.8°N, West 15.5°E, East 18.5°E

Option 2: Single large query covering both regions

North 51.7°N, South 39.8°N, West -0.5°W, East 18.5°E
This creates a massive grid covering most of Europe, resulting in much larger files

I'd recommend Option 1 as it's more efficient and gives you focused data for each region.
Puglia Grid Coordinates
Puglia (Apulia) occupies the "heel" of Italy's boot:

North: 41.9°N (around Foggia)
South: 39.8°N (around Santa Maria di Leuca)
West: 15.5°E (around Bari)
East: 18.5°E (Adriatic coast)

Data Extraction from GRIB Files
Once you have the .grib files, extract data using these approaches:
1. Point-based comparison:

Extract data for specific coordinates (e.g., London center: 51.5°N, -0.1°W; Bari: 41.1°N, 16.9°E)
Use tools like grib_get_data or Python libraries (xarray, pygrib)

2. Area-averaged comparison:

Calculate spatial averages for each region
This smooths out local variations and gives regional climate signals

3. Representative city comparison:

London: Extract nearest grid point to 51.5°N, -0.1°W
Puglia: Extract for major cities like Bari (41.1°N, 16.9°E) or Lecce (40.4°N, 18.2°E)

> grib_file = "./data/ERA5/ERA5_hourly_data_on_single_levels_from_1940_to_present.grib"  # Replace with your GRIB file path
- output_csv = "./output/ERA5_hourly_data_on_single_levels_from_1940_to_present.csv"  # Replace with desired output path
- hourly data from 2022 to 2024

> 

### GRIB to csv conversion

In [2]:
# Version 2: first working version
import warnings
warnings.filterwarnings("ignore")

import cfgrib
import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def grib_to_csv(grib_file_path,
                city_lat,
                city_lon,
                output_csv_path='./output/',
                city_name = 'london',
                print_debug=False,
                ):
    """
    Read GRIB file, extract data for London coordinates, and save as CSV.
    
    Parameters:
    grib_file_path (str): Path to the GRIB file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    if print_debug:
        print(f"Reading GRIB file: {grib_file_path}")
    
    try:
        # Open GRIB file with xarray
        print('breakpoint 0') 
        datasets = cfgrib.open_datasets(grib_file_path)
        for i, ds in enumerate(datasets):
            print(f"Dataset {i}:")
            print(f"Variables: {list(ds.data_vars.keys())}")
            print(f"Time range: {ds.time.min().values} to {ds.time.max().values}")
            print("---")
        
        ds = xr.open_dataset(grib_file_path, engine='cfgrib')
        print('breakpoint 1')        
        # Check longitude format (0-360 vs -180-180)
        city_min = ds.longitude.min().values
        city_max = ds.longitude.max().values
        if print_debug:
            print(f"Longitude range: {city_min} to {city_max}")
        
        target_city = city_lon
        if print_debug:
            print(f"Using -180-180 longitude format: {target_city}")
        
        # Find nearest grid point to London
        lat_idx = np.abs(ds.latitude - city_lat).argmin()
        lon_idx = np.abs(ds.longitude - target_city).argmin()
        
        actual_lat = ds.latitude[lat_idx].values
        actual_lon = ds.longitude[lon_idx].values
        
        if print_debug:
            print(f"Target coordinates: ({city_lat}, {target_city})")
            print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        
        # Extract data for London coordinates
        city_data = ds.isel(latitude=lat_idx, longitude=lon_idx)
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in city_data.data_vars:
            var_data = city_data[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Create DataFrame with time index
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            elif 'valid_time' in var_data.dims:
                # Handle valid_time dimension
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            else:
                # Single value variables
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Merge on common columns (time, latitude, longitude)
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols:
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # If no common columns, concatenate side by side
                    df = pd.concat([df, df_var], axis=1)
        else:
            df = df_list[0]
        
        df['target_latitude'] = city_lat
        df['target_longitude'] = target_city
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            df = df.sort_values(time_cols[0])
            if print_debug:
                print(f"****\n - inside csv Time range: {df[time_col].min()} to {df[time_col].max()}")
            min_string = df[time_col].min().strftime('%Y%m%d')
            max_string = df[time_col].max().strftime('%Y%m%d')
            if print_debug:
                print(f" - min time as string: {min_string}")
                print(f" - max time as string: {max_string}")
            output_csv_name = list(ds.data_vars)[0] + '_' + min_string + '_' + max_string
            if print_debug:
                print(output_csv_name)
        
        if print_debug:
            print(f"DataFrame shape: {df.shape}")
            print(f"DataFrame columns: {list(df.columns)}")
            print("grib to csv")
            print(f"ds.data_vars dtype: {type(ds.data_vars)}, df.data_vars name: {list(ds.data_vars)[0]}")
        # print("\nFirst few rows:")
        # print(df.head())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(grib_file_path).stem + "_city_data.csv"
        
        output_csv_path = output_csv_path + city_name + '_' + output_csv_name + '.csv'
        if print_debug:
            print(f"Saving DataFrame to: {output_csv_path}")
        

        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing GRIB file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray cfgrib pandas numpy")
        raise

In [3]:
def process_grib_file(grib_file_path,
                      output_csv_path='./output/',
                      city_name='london',
                      city_lat=51.5074,  # Default to London latitude
                      city_lon=-0.1278,
                      print_debug = False):  # Default to London longitude
    
    grib_file = grib_file_path  # Replace with your GRIB file path
    output_csv_city = city_name  # Replace with desired output path
    output_csv_path = output_csv_path  # Directory to save the output CSV

    # Check if file exists
    if not Path(grib_file).exists():
        print(f"GRIB file not found: {grib_file}")
        print("Please update the 'grib_file' variable with the correct path to your GRIB file.")
        return
    
    try:
        df = grib_to_csv(grib_file,
                         city_lat=city_lat,
                         city_lon=-city_lon, 
                         output_csv_path=output_csv_path, 
                         city_name=output_csv_city, 
                         print_debug=print_debug)
        
        # Display summary statistics
        if print_debug:
            print("\n" + "="*50)
            print("SUMMARY")
            print("="*50)
            print(f"Total records: {len(df)}")
        
        # Show time range if available
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            if print_debug:
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
        
        if print_debug:
            print(f"Variables extracted: {[col for col in df.columns if col not in ['latitude', 'longitude', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude'] + time_cols]}")
        
        return df
        
    except Exception as e:
        print(f"Failed to process GRIB file: {e}")


In [4]:
# Process the file
# Bari 41.14043890868558, 16.86619063277282
# Lecce 40.35654102220363, 18.17311913754415
# London 51.5074, -0.1278

file_path = './data/ERA5/data_london_2mt_2005_2010.grib'
output_path = './output/'
city_name = 'london'
city_lat = 51.5074  # London latitude
city_lon = -0.1278  # London longitude

df_test = process_grib_file(grib_file_path = file_path,
                            output_csv_path=output_path,
                            city_name=city_name,
                            city_lat=city_lat,
                            city_lon=city_lon)

breakpoint 0
Dataset 0:
Variables: ['t2m']
Time range: 2005-01-01T00:00:00.000000000 to 2010-12-31T23:00:00.000000000
---
Dataset 1:
Variables: ['tp']
Time range: 2004-12-31T18:00:00.000000000 to 2010-12-31T18:00:00.000000000
---


skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "c:\ProgramData\anaconda3\Lib\site-packages\cfgrib\dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1104537600, 1104541200, 1104544800, ..., 1293829200, 1293832800,
       1293836400])) new_value=Variable(dimensions=('time',), data=array([1104516000, 1104559200, 1104602400, ..., 1293732000, 1293775200,
       1293818400]))


breakpoint 1

Data saved to: ./output/london_t2m_20050101_20101231.csv


In [5]:
df_test.head()

,time,number,step,surface,latitude,longitude,valid_time,t2m,target_latitude,target_longitude
0,2005-01-01 00:00:00,0,0 days,0.0,51.55,0.25,2005-01-01 00:00:00,279.231567,51.5074,0.1278
1,2005-01-01 01:00:00,0,0 days,0.0,51.55,0.25,2005-01-01 01:00:00,279.666870,51.5074,0.1278
2,2005-01-01 02:00:00,0,0 days,0.0,51.55,0.25,2005-01-01 02:00:00,279.765686,51.5074,0.1278
3,2005-01-01 03:00:00,0,0 days,0.0,51.55,0.25,2005-01-01 03:00:00,279.600403,51.5074,0.1278
4,2005-01-01 04:00:00,0,0 days,0.0,51.55,0.25,2005-01-01 04:00:00,279.259583,51.5074,0.1278


In [ ]:
# Process the file
# Bari 41.14043890868558, 16.86619063277282
# Lecce 40.35654102220363, 18.17311913754415
# London 51.5074, -0.1278


file_path = './data/ERA5/data_london_2mt_2005_2010.grib'
output_path = './output/'
city_name = 'london'
city_lat = 51.5074  # London latitude
city_lon = -0.1278  # London longitude

df_test= process_grib_file(grib_file_path = file_path,
                            output_csv_path=output_path,
                            city_name=city_name,
                            city_lat=city_lat,
                            city_lon=city_lon)

In [6]:
input_path = './data/ERA5/'
file_names = ['london_tp_2014_2021.grib','london_tp_2022_2024.grib']

for file_name in file_names:
    print(input_path + file_name)

df_data = []
for file_name in file_names:
    df = process_grib_file(grib_file_path=input_path + file_name,
                           output_csv_path=output_path,
                           city_name=city_name,
                           city_lat=city_lat,
                           city_lon=city_lon)
    print(df.shape)
    df_data.append(df)
# Concatenate all DataFrames
df_combined = pd.concat(df_data, ignore_index=True)
print(df_combined.shape)

./data/ERA5/london_tp_2014_2021.grib
./data/ERA5/london_tp_2022_2024.grib
breakpoint 0
Dataset 0:
Variables: ['tp']
Time range: 2013-12-31T18:00:00.000000000 to 2021-12-31T18:00:00.000000000
---
breakpoint 1

Data saved to: ./output/london_tp_20131231_20211231.csv
(70140, 10)
breakpoint 0
Dataset 0:
Variables: ['tp']
Time range: 2021-12-31T18:00:00.000000000 to 2024-12-31T18:00:00.000000000
---
breakpoint 1

Data saved to: ./output/london_tp_20211231_20241231.csv
(26316, 10)
(96456, 10)


In [14]:
df_combined.sort_values(by='valid_time', inplace=True)
df_combined.reset_index(drop=True, inplace=True)

In [20]:
df_combined[df_combined.tp.isna()]

,time,step,number,surface,latitude,longitude,valid_time,tp,target_latitude,target_longitude
0,2013-12-31 18:00:00,0 days 01:00:00,0,0.0,51.4,0.05,2013-12-31 19:00:00,NaN,51.5074,0.1278
1,2013-12-31 18:00:00,0 days 02:00:00,0,0.0,51.4,0.05,2013-12-31 20:00:00,NaN,51.5074,0.1278
2,2013-12-31 18:00:00,0 days 03:00:00,0,0.0,51.4,0.05,2013-12-31 21:00:00,NaN,51.5074,0.1278
3,2013-12-31 18:00:00,0 days 04:00:00,0,0.0,51.4,0.05,2013-12-31 22:00:00,NaN,51.5074,0.1278
4,2013-12-31 18:00:00,0 days 05:00:00,0,0.0,51.4,0.05,2013-12-31 23:00:00,NaN,51.5074,0.1278
70128,2021-12-31 18:00:00,0 days 01:00:00,0,0.0,51.6,0.05,2021-12-31 19:00:00,NaN,51.5074,0.1278
70130,2021-12-31 18:00:00,0 days 02:00:00,0,0.0,51.6,0.05,2021-12-31 20:00:00,NaN,51.5074,0.1278
70132,2021-12-31 18:00:00,0 days 03:00:00,0,0.0,51.6,0.05,2021-12-31 21:00:00,NaN,51.5074,0.1278
70134,2021-12-31 18:00:00,0 days 04:00:00,0,0.0,51.6,0.05,2021-12-31 22:00:00,NaN,51.5074,0.1278
70136,2021-12-31 18:00:00,0 days 05:00:00,0,0.0,51.6,0.05,2021-12-31 23:00:00,NaN,51.5074,0.1278


In [17]:
df_combined.head()

,time,step,number,surface,latitude,longitude,valid_time,tp,target_latitude,target_longitude
0,2013-12-31 18:00:00,0 days 01:00:00,0,0.0,51.4,0.05,2013-12-31 19:00:00,NaN,51.5074,0.1278
1,2013-12-31 18:00:00,0 days 02:00:00,0,0.0,51.4,0.05,2013-12-31 20:00:00,NaN,51.5074,0.1278
2,2013-12-31 18:00:00,0 days 03:00:00,0,0.0,51.4,0.05,2013-12-31 21:00:00,NaN,51.5074,0.1278
3,2013-12-31 18:00:00,0 days 04:00:00,0,0.0,51.4,0.05,2013-12-31 22:00:00,NaN,51.5074,0.1278
4,2013-12-31 18:00:00,0 days 05:00:00,0,0.0,51.4,0.05,2013-12-31 23:00:00,NaN,51.5074,0.1278


In [11]:
df_combined.valid_time.max()

Timestamp('2025-01-01 06:00:00')

In [ ]:
df_combined.valid_time.min()

In [ ]:
df_combined.valid_time.max()

In [ ]:
file_path = './data/ERA5/data_puglia_tp_2025.grib'
output_path = './output/'
city_name = 'bari'
city_lat = 41.14043890868558  # Bari latitude
city_lon = 16.86619063277282  # Bari longitude

df_test_bari = process_grib_file(grib_file_path = file_path,
                            output_csv_path=output_path,
                            city_name=city_name,
                            city_lat=city_lat,
                            city_lon=city_lon)

In [ ]:
# Lecce 40.35654102220363, 18.17311913754415

file_path = './data/ERA5/data_puglia_tp_2025.grib'
output_path = './output/'
city_name = 'lecce'
city_lat = 40.35654102220363  # Lecce latitude
city_lon = 18.17311913754415  # Lecce longitude

df_test_lecce = process_grib_file(grib_file_path = file_path,
                                  output_csv_path=output_path,
                                  city_name=city_name,
                                  city_lat=city_lat,
                                  city_lon=city_lon)

In [ ]:
# Working for London coordinates

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def grib_to_csv_london(grib_file_path, output_csv_path=None):
    """
    Read GRIB file, extract data for London coordinates, and save as CSV.
    
    Parameters:
    grib_file_path (str): Path to the GRIB file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278  # Note: GRIB files often use 0-360 longitude
    london_lon_360 = 359.8722  # London longitude in 0-360 format
    
    print(f"Reading GRIB file: {grib_file_path}")
    
    try:
        # Open GRIB file with xarray
        ds = xr.open_dataset(grib_file_path, engine='cfgrib')
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Check longitude format (0-360 vs -180-180)
        lon_min = ds.longitude.min().values
        lon_max = ds.longitude.max().values
        print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Use appropriate London longitude based on the dataset's longitude format
        if lon_max > 180:
            target_lon = london_lon_360
            print(f"Using 0-360 longitude format: {target_lon}")
        else:
            target_lon = london_lon
            print(f"Using -180-180 longitude format: {target_lon}")
        
        # Find nearest grid point to London
        lat_idx = np.abs(ds.latitude - london_lat).argmin()
        lon_idx = np.abs(ds.longitude - target_lon).argmin()
        
        actual_lat = ds.latitude[lat_idx].values
        actual_lon = ds.longitude[lon_idx].values
        
        print(f"Target coordinates: ({london_lat}, {target_lon})")
        print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        
        # Extract data for London coordinates
        london_data = ds.isel(latitude=lat_idx, longitude=lon_idx)
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Create DataFrame with time index
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            elif 'valid_time' in var_data.dims:
                # Handle valid_time dimension
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            else:
                # Single value variables
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Merge on common columns (time, latitude, longitude)
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols:
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # If no common columns, concatenate side by side
                    df = pd.concat([df, df_var], axis=1)
        else:
            df = df_list[0]
        
        # Add metadata columns
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = target_lon
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            df = df.sort_values(time_cols[0])
        
        print(f"DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        print("\nFirst few rows:")
        print(df.head())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(grib_file_path).stem + "_london_data.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing GRIB file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray cfgrib pandas numpy")
        raise

def main():
    # Example usage
    grib_file = "./data/ERA5/ERA5_hourly_data_on_single_levels_from_2014_to_2021.grib"  # Replace with your GRIB file path
    output_csv = "./output/ERA5_hourly_data_on_single_levels_from_2014_to_2021.csv"  # Replace with desired output path
    
    # Check if file exists
    if not Path(grib_file).exists():
        print(f"GRIB file not found: {grib_file}")
        print("Please update the 'grib_file' variable with the correct path to your GRIB file.")
        return
    
    # Process the file
    try:
        df = grib_to_csv_london(grib_file, output_csv)
        
        # Display summary statistics
        print("\n" + "="*50)
        print("SUMMARY")
        print("="*50)
        print(f"Total records: {len(df)}")
        
        # Show time range if available
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
        
        print(f"Variables extracted: {[col for col in df.columns if col not in ['latitude', 'longitude', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude'] + time_cols]}")
        
        # Load the CSV back into pandas to verify
        print("\n" + "="*50)
        print("VERIFICATION - Loading CSV back into pandas:")
        print("="*50)
        df_loaded = pd.read_csv(output_csv)
        print(f"Loaded DataFrame shape: {df_loaded.shape}")
        print("DataFrame info:")
        print(df_loaded.info())
        
    except Exception as e:
        print(f"Failed to process GRIB file: {e}")

if __name__ == "__main__":
    main()

In [ ]:
df_data = pd.read_csv("./output/ERA5_hourly_data_on_single_levels_from_2014_to_2021.csv")
print(df_data.shape)
df_data.head()

In [ ]:
140-80

In [ ]:
df_data = pd.read_csv("./output/ERA5_hourly_data_on_single_levels_from_1940_to_present.csv")
df_data.tail()

In [ ]:
df_data.shape

In [ ]:
df_data['year'] = pd.to_datetime(df_data['time'], errors='coerce').dt.year


In [ ]:
(df_data
 .groupby('year', observed=True)['tp']
 .sum()
)

### NC files

In [ ]:
# V4 

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Define region around London (in degrees)
        region_size = 0.5  # degrees (approximately 50km at London's latitude)
        
        # Define bounding box around London
        lat_min = london_lat - region_size
        lat_max = london_lat + region_size
        lon_min = london_lon - region_size
        lon_max = london_lon + region_size
        
        print(f"\nExtracting region around London:")
        print(f"Latitude range: {lat_min:.3f} to {lat_max:.3f}")
        print(f"Longitude range: {lon_min:.3f} to {lon_max:.3f}")
        
        # Find grid indices for the region
        if lat_coord in ds.coords and lon_coord in ds.coords:
            # Handle 2D coordinate arrays (common in projected datasets)
            lat_data = ds.coords[lat_coord]
            lon_data = ds.coords[lon_coord]
            
            if lat_data.ndim == 2 and lon_data.ndim == 2:
                # 2D coordinate arrays - find region
                print("Using 2D coordinate arrays to find region")
                
                # Find points within the bounding box
                within_region = ((lat_data >= lat_min) & (lat_data <= lat_max) & 
                               (lon_data >= lon_min) & (lon_data <= lon_max))
                
                # Find the bounding indices of the region
                y_indices, x_indices = np.where(within_region)
                
                if len(y_indices) == 0:
                    print("No grid points found in the specified region!")
                    return None
                
                y_min_idx = y_indices.min()
                y_max_idx = y_indices.max()
                x_min_idx = x_indices.min()
                x_max_idx = x_indices.max()
                
                print(f"Found {len(y_indices)} grid points in region")
                print(f"Grid index ranges: y={y_min_idx}-{y_max_idx}, x={x_min_idx}-{x_max_idx}")
                
                # Store the slice ranges for extraction
                y_slice = slice(y_min_idx, y_max_idx + 1)
                x_slice = slice(x_min_idx, x_max_idx + 1)
                
                # Get actual coordinate bounds of extracted region
                region_lat_min = lat_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].min().values
                region_lat_max = lat_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].max().values
                region_lon_min = lon_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].min().values
                region_lon_max = lon_data[y_min_idx:y_max_idx+1, x_min_idx:x_max_idx+1].max().values
                
                print(f"Actual extracted region:")
                print(f"  Latitude: {region_lat_min:.3f} to {region_lat_max:.3f}")
                print(f"  Longitude: {region_lon_min:.3f} to {region_lon_max:.3f}")
                
            else:
                # 1D coordinate arrays
                lat_mask = (lat_data >= lat_min) & (lat_data <= lat_max)
                lon_mask = (lon_data >= lon_min) & (lon_data <= lon_max)
                
                y_indices = np.where(lat_mask)[0]
                x_indices = np.where(lon_mask)[0]
                
                if len(y_indices) == 0 or len(x_indices) == 0:
                    print("No grid points found in the specified region!")
                    return None
                
                y_slice = slice(y_indices.min(), y_indices.max() + 1)
                x_slice = slice(x_indices.min(), x_indices.max() + 1)
                
                print(f"Grid slices: y={y_slice}, x={x_slice}")
        else:
            print("Could not find coordinate values for region extraction")
            return None
        
        # Extract data for London region
        # Use the projection coordinate names for selection
        proj_y_coord = 'projection_y_coordinate'
        proj_x_coord = 'projection_x_coordinate'
        
        selection_dict = {proj_y_coord: y_slice, proj_x_coord: x_slice}
        london_region_data = ds.isel(selection_dict)
        
        print(f"\nExtracted region data shape: {dict(london_region_data.sizes)}")
        
        print(f"\nExtracted data variables: {list(london_region_data.data_vars)}")
        
        # Convert to pandas DataFrame - simplified approach
        print(f"\nExtracting data for London...")
        
        # Create a simple DataFrame with the main data
        data_dict = {}
        
        # Extract main variables (skip coordinate and bounds variables)
        main_vars = [var for var in london_region_data.data_vars 
                    if not any(skip in var for skip in ['_bnds', 'transverse_mercator', 'bounds'])]
        
        print(f"Main data variables to extract: {main_vars}")
        
        for var_name in main_vars:
            var_data = london_region_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Extract the actual values
            if var_data.size == 1:
                # Single value
                data_dict[var_name] = [float(var_data.values)]
            elif time_coord and time_coord in var_data.dims:
                # Time series data
                values = var_data.values
                if values.ndim > 1:
                    values = values.flatten()
                data_dict[var_name] = values.tolist()
            else:
                # Other cases
                values = var_data.values
                if hasattr(values, 'flatten'):
                    values = values.flatten()
                data_dict[var_name] = [float(values)] if np.isscalar(values) else values.tolist()
            
            # Add metadata
            if hasattr(var_data, 'attrs'):
                units = var_data.attrs.get('units', 'unknown')
                long_name = var_data.attrs.get('long_name', var_name)
                data_dict[f'{var_name}_units'] = [units] * len(data_dict[var_name])
                data_dict[f'{var_name}_long_name'] = [long_name] * len(data_dict[var_name])
        
        # Add time information if available
        if time_coord and time_coord in london_region_data.coords:
            time_values = london_region_data.coords[time_coord].values
            if hasattr(time_values, 'flatten'):
                time_values = time_values.flatten()
            data_dict[time_coord] = time_values.tolist() if hasattr(time_values, 'tolist') else [time_values]
        
        # Create DataFrame
        # First, make sure all lists have the same length
        max_length = max(len(v) if isinstance(v, list) else 1 for v in data_dict.values())
        
        for key, value in data_dict.items():
            if isinstance(value, list) and len(value) == 1 and max_length > 1:
                data_dict[key] = value * max_length
        
        df = pd.DataFrame(data_dict)
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_region_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "./output/london_region_rainfall_1961_1990_V4.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total grid points extracted: {len(df)}")
            
            # Show spatial extent
            if 'latitude' in df.columns and 'longitude' in df.columns:
                print(f"Latitude range: {df['latitude'].min():.3f} to {df['latitude'].max():.3f}")
                print(f"Longitude range: {df['longitude'].min():.3f} to {df['longitude'].max():.3f}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nRegional precipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
                print(f"Grid points with data: {df[precip_cols[0]].notna().sum()}/{len(df)}")
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

In [ ]:
# V3

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Find nearest grid point to London
        if lat_coord in ds.coords and lon_coord in ds.coords:
            # Handle 2D coordinate arrays (common in projected datasets)
            lat_data = ds.coords[lat_coord]
            lon_data = ds.coords[lon_coord]
            
            if lat_data.ndim == 2 and lon_data.ndim == 2:
                # 2D coordinate arrays - find nearest point in 2D space
                print("Using 2D coordinate arrays to find nearest point")
                
                # Calculate distance from London to all grid points
                lat_diff = lat_data - london_lat
                lon_diff = lon_data - london_lon
                distances = np.sqrt(lat_diff**2 + lon_diff**2)
                
                # Find indices of minimum distance
                min_idx = np.unravel_index(distances.argmin(), distances.shape)
                y_idx, x_idx = min_idx
                
                actual_lat = lat_data[y_idx, x_idx].values
                actual_lon = lon_data[y_idx, x_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
                print(f"Grid indices: y={y_idx}, x={x_idx}")
                
                # Use the projection coordinate indices for selection
                lat_idx = y_idx  # This corresponds to projection_y_coordinate index
                lon_idx = x_idx  # This corresponds to projection_x_coordinate index
                
            else:
                # 1D coordinate arrays
                lat_idx = np.abs(lat_data - london_lat).argmin()
                lon_idx = np.abs(lon_data - london_lon).argmin()
                
                actual_lat = lat_data[lat_idx].values
                actual_lon = lon_data[lon_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        else:
            print("Could not find coordinate values for nearest point calculation")
            return None
        
        # Extract data for London coordinates
        # Use the projection coordinate names for selection
        proj_y_coord = 'projection_y_coordinate'
        proj_x_coord = 'projection_x_coordinate'
        
        selection_dict = {proj_y_coord: lat_idx, proj_x_coord: lon_idx}
        london_data = ds.isel(selection_dict)
        
        print(f"\nExtracted data variables: {list(london_data.data_vars)}")
        
        # Convert to pandas DataFrame - simplified approach
        print(f"\nExtracting data for London...")
        
        # Create a simple DataFrame with the main data
        data_dict = {}
        
        # Extract main variables (skip coordinate and bounds variables)
        main_vars = [var for var in london_data.data_vars 
                    if not any(skip in var for skip in ['_bnds', 'transverse_mercator', 'bounds'])]
        
        print(f"Main data variables to extract: {main_vars}")
        
        for var_name in main_vars:
            var_data = london_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Extract the actual values
            if var_data.size == 1:
                # Single value
                data_dict[var_name] = [float(var_data.values)]
            elif time_coord and time_coord in var_data.dims:
                # Time series data
                values = var_data.values
                if values.ndim > 1:
                    values = values.flatten()
                data_dict[var_name] = values.tolist()
            else:
                # Other cases
                values = var_data.values
                if hasattr(values, 'flatten'):
                    values = values.flatten()
                data_dict[var_name] = [float(values)] if np.isscalar(values) else values.tolist()
            
            # Add metadata
            if hasattr(var_data, 'attrs'):
                units = var_data.attrs.get('units', 'unknown')
                long_name = var_data.attrs.get('long_name', var_name)
                data_dict[f'{var_name}_units'] = [units] * len(data_dict[var_name])
                data_dict[f'{var_name}_long_name'] = [long_name] * len(data_dict[var_name])
        
        # Add time information if available
        if time_coord and time_coord in london_data.coords:
            time_values = london_data.coords[time_coord].values
            if hasattr(time_values, 'flatten'):
                time_values = time_values.flatten()
            data_dict[time_coord] = time_values.tolist() if hasattr(time_values, 'tolist') else [time_values]
        
        # Create DataFrame
        # First, make sure all lists have the same length
        max_length = max(len(v) if isinstance(v, list) else 1 for v in data_dict.values())
        
        for key, value in data_dict.items():
            if isinstance(value, list) and len(value) == 1 and max_length > 1:
                data_dict[key] = value * max_length
        
        df = pd.DataFrame(data_dict)
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "./output/london_rainfall_1961_1990_V3.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total records: {len(df)}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nPrecipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

In [ ]:
# Not working for datatype mismatch issues

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Find nearest grid point to London
        if lat_coord in ds.coords and lon_coord in ds.coords:
            # Handle 2D coordinate arrays (common in projected datasets)
            lat_data = ds.coords[lat_coord]
            lon_data = ds.coords[lon_coord]
            
            if lat_data.ndim == 2 and lon_data.ndim == 2:
                # 2D coordinate arrays - find nearest point in 2D space
                print("Using 2D coordinate arrays to find nearest point")
                
                # Calculate distance from London to all grid points
                lat_diff = lat_data - london_lat
                lon_diff = lon_data - london_lon
                distances = np.sqrt(lat_diff**2 + lon_diff**2)
                
                # Find indices of minimum distance
                min_idx = np.unravel_index(distances.argmin(), distances.shape)
                y_idx, x_idx = min_idx
                
                actual_lat = lat_data[y_idx, x_idx].values
                actual_lon = lon_data[y_idx, x_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
                print(f"Grid indices: y={y_idx}, x={x_idx}")
                
                # Use the projection coordinate indices for selection
                lat_idx = y_idx  # This corresponds to projection_y_coordinate index
                lon_idx = x_idx  # This corresponds to projection_x_coordinate index
                
            else:
                # 1D coordinate arrays
                lat_idx = np.abs(lat_data - london_lat).argmin()
                lon_idx = np.abs(lon_data - london_lon).argmin()
                
                actual_lat = lat_data[lat_idx].values
                actual_lon = lon_data[lon_idx].values
                
                print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
                print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        else:
            print("Could not find coordinate values for nearest point calculation")
            return None
        
        # Extract data for London coordinates
        # Use the projection coordinate names for selection
        proj_y_coord = 'projection_y_coordinate'
        proj_x_coord = 'projection_x_coordinate'
        
        selection_dict = {proj_y_coord: lat_idx, proj_x_coord: lon_idx}
        london_data = ds.isel(selection_dict)
        
        print(f"\nExtracted data variables: {list(london_data.data_vars)}")
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Convert to DataFrame
            if time_coord and time_coord in var_data.dims:
                # Time series data
                df_var = var_data.to_dataframe().reset_index()
                
                # Rename the variable column to be more descriptive
                if var_name in df_var.columns:
                    # Keep original variable name but add units if available
                    units = var_data.attrs.get('units', '')
                    long_name = var_data.attrs.get('long_name', var_name)
                    
                    col_name = var_name
                    if units:
                        col_name += f"_{units}".replace(" ", "_")
                    
                    df_var = df_var.rename(columns={var_name: col_name})
                    
                    # Add metadata as separate columns
                    df_var[f'{var_name}_long_name'] = long_name
                    df_var[f'{var_name}_units'] = units
                
                df_list.append(df_var)
            else:
                # Single value or no time dimension
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                
                # Add metadata
                if hasattr(var_data, 'attrs'):
                    for attr_name, attr_value in var_data.attrs.items():
                        df_var[f'{var_name}_{attr_name}'] = str(attr_value)
                
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Find common columns for merging
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols and len(common_cols) > 1:  # More than just the variable column
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # Concatenate side by side if same length
                    if len(df) == len(df_var):
                        df = pd.concat([df, df_var], axis=1)
                    else:
                        print(f"Warning: Cannot merge {df_var.columns} - different lengths")
        else:
            df = df_list[0] if df_list else pd.DataFrame()
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "./output/london_rainfall_1961_1990_V2.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total records: {len(df)}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nPrecipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

from pathlib import Path

def grib_to_csv_london(grib_file_path, output_csv_path=None):
    """
    Read GRIB file, extract data for London coordinates, and save as CSV.
    
    Parameters:
    grib_file_path (str): Path to the GRIB file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278  # Note: GRIB files often use 0-360 longitude
    london_lon_360 = 359.8722  # London longitude in 0-360 format
    
    print(f"Reading GRIB file: {grib_file_path}")
    
    try:
        # Open GRIB file with xarray
        ds = xr.open_dataset(grib_file_path, engine='cfgrib')
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Check longitude format (0-360 vs -180-180)
        lon_min = ds.longitude.min().values
        lon_max = ds.longitude.max().values
        print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Use appropriate London longitude based on the dataset's longitude format
        if lon_max > 180:
            target_lon = london_lon_360
            print(f"Using 0-360 longitude format: {target_lon}")
        else:
            target_lon = london_lon
            print(f"Using -180-180 longitude format: {target_lon}")
        
        # Find nearest grid point to London
        lat_idx = np.abs(ds.latitude - london_lat).argmin()
        lon_idx = np.abs(ds.longitude - target_lon).argmin()
        
        actual_lat = ds.latitude[lat_idx].values
        actual_lon = ds.longitude[lon_idx].values
        
        print(f"Target coordinates: ({london_lat}, {target_lon})")
        print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        
        # Extract data for London coordinates
        london_data = ds.isel(latitude=lat_idx, longitude=lon_idx)
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Create DataFrame with time index
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            elif 'valid_time' in var_data.dims:
                # Handle valid_time dimension
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            else:
                # Single value variables
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Merge on common columns (time, latitude, longitude)
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols:
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # If no common columns, concatenate side by side
                    df = pd.concat([df, df_var], axis=1)
        else:
            df = df_list[0]
        
        # Add metadata columns
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = target_lon
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            df = df.sort_values(time_cols[0])
        
        print(f"DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        print("\nFirst few rows:")
        print(df.head())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(grib_file_path).stem + "_london_data.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing GRIB file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray cfgrib pandas numpy")
        raise

def main():
    # Example usage
    grib_file = "./data/ERA5/ERA5_hourly_data_on_single_levels_from_1940_to_present.grib"  # Replace with your GRIB file path
    output_csv = "./output/london_weather_data.csv"  # Replace with desired output path
    
    # Check if file exists
    if not Path(grib_file).exists():
        print(f"GRIB file not found: {grib_file}")
        print("Please update the 'grib_file' variable with the correct path to your GRIB file.")
        return
    
    # Process the file
    try:
        df = grib_to_csv_london(grib_file, output_csv)
        
        # Display summary statistics
        print("\n" + "="*50)
        print("SUMMARY")
        print("="*50)
        print(f"Total records: {len(df)}")
        
        # Show time range if available
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
        
        print(f"Variables extracted: {[col for col in df.columns if col not in ['latitude', 'longitude', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude'] + time_cols]}")
        
        # Load the CSV back into pandas to verify
        print("\n" + "="*50)
        print("VERIFICATION - Loading CSV back into pandas:")
        print("="*50)
        df_loaded = pd.read_csv(output_csv)
        print(f"Loaded DataFrame shape: {df_loaded.shape}")
        print("DataFrame info:")
        print(df_loaded.info())
        
    except Exception as e:
        print(f"Failed to process GRIB file: {e}")

if __name__ == "__main__":
    main()

In [ ]:
df_data = pd.read_csv("./output/london_weather_data.csv")
df_data.head()

In [ ]:
df_data['year'] = df_data['datetime'].dt.year
df_data['year'].unique()

In [ ]:
(df_data
 .groupby('year', observed=True)['tp']
 .sum()
)


### Functions Backup

In [ ]:
# Version 1: first working version

import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def grib_to_csv_city(grib_file_path, 
                     city_lat, 
                     city_lon,
                     output_csv_path='./output/',
                     city_name = 'london', 
                     ):
    """
    Read GRIB file, extract data for London coordinates, and save as CSV.
    
    Parameters:
    grib_file_path (str): Path to the GRIB file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # # London coordinates
    # london_lat = 51.5074
    # london_lon = -0.1278  # Note: GRIB files often use 0-360 longitude
    # london_lon_360 = 359.8722  # London longitude in 0-360 format
    
    print(f"Reading GRIB file: {grib_file_path}")
    
    try:
        # Open GRIB file with xarray
        ds = xr.open_dataset(grib_file_path, engine='cfgrib')
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Check longitude format (0-360 vs -180-180)
        city_min = ds.longitude.min().values
        city_max = ds.longitude.max().values
        print(f"Longitude range: {city_min} to {city_max}")
        
        # Use appropriate London longitude based on the dataset's longitude format
        # if city_max > 180:
        #     target_lon = london_lon_360
        #     print(f"Using 0-360 longitude format: {target_lon}")
        # else:
        target_city = city_lon
        print(f"Using -180-180 longitude format: {target_city}")
        
        # Find nearest grid point to London
        lat_idx = np.abs(ds.latitude - city_lat).argmin()
        lon_idx = np.abs(ds.longitude - target_city).argmin()
        
        actual_lat = ds.latitude[lat_idx].values
        actual_lon = ds.longitude[lon_idx].values
        
        print(f"Target coordinates: ({city_lat}, {target_city})")
        print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        
        # Extract data for London coordinates
        city_data = ds.isel(latitude=lat_idx, longitude=lon_idx)
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in city_data.data_vars:
            var_data = city_data[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Create DataFrame with time index
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            elif 'valid_time' in var_data.dims:
                # Handle valid_time dimension
                df_var = var_data.to_dataframe().reset_index()
                df_var = df_var.rename(columns={var_name: var_name})
                df_list.append(df_var)
            else:
                # Single value variables
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Merge on common columns (time, latitude, longitude)
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols:
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # If no common columns, concatenate side by side
                    df = pd.concat([df, df_var], axis=1)
        else:
            df = df_list[0]
        
        # Add metadata columns
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = city_lat
        df['target_longitude'] = target_city
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            df = df.sort_values(time_cols[0])
            print(f"****\n - inside csv Time range: {df[time_col].min()} to {df[time_col].max()}")
            min_string = df[time_col].min().strftime('%Y%m%d')
            max_string = df[time_col].max().strftime('%Y%m%d')
            print(f" - min time as string: {min_string}")
            print(f" - max time as string: {max_string}")
            output_csv_name = list(ds.data_vars)[0] + '_' + min_string + '_' + max_string
            print(output_csv_name)
        
        print(f"DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        print("grib to csv")
        print(f"ds.data_vars dtype: {type(ds.data_vars)}, df.data_vars name: {list(ds.data_vars)[0]}")
        # print("\nFirst few rows:")
        # print(df.head())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(grib_file_path).stem + "_city_data.csv"
        
        output_csv_path = output_csv_path + city_name + '_' + output_csv_name + '.csv'
        print(f"Saving DataFrame to: {output_csv_path}")
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing GRIB file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray cfgrib pandas numpy")
        raise

In [ ]:
# Version 1

def process_grib_file(grib_file_path,
                      output_csv_path='./output/',
                      city_name='london',
                      city_lat=51.5074,  # Default to London latitude
                      city_lon=-0.1278):  # Default to London longitude
    
    # Example usage
    grib_file = grib_file_path  # Replace with your GRIB file path
    output_csv_city = city_name  # Replace with desired output path
    output_csv_path = output_csv_path  # Directory to save the output CSV

    # Check if file exists
    if not Path(grib_file).exists():
        print(f"GRIB file not found: {grib_file}")
        print("Please update the 'grib_file' variable with the correct path to your GRIB file.")
        return
    
    try:
        df = grib_to_csv_city(grib_file, 
                              city_lat=city_lat, 
                              city_lon=-city_lon, output_csv_path=output_csv_path, city_name=output_csv_city)
        
        # Display summary statistics
        print("\n" + "="*50)
        print("SUMMARY")
        print("="*50)
        print(f"Total records: {len(df)}")
        
        # Show time range if available
        time_cols = [col for col in df.columns if 'time' in col.lower()]
        if time_cols:
            time_col = time_cols[0]
            print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
        
        print(f"Variables extracted: {[col for col in df.columns if col not in ['latitude', 'longitude', 'actual_latitude', 'actual_longitude', 'target_latitude', 'target_longitude'] + time_cols]}")
        
        # # Load the CSV back into pandas to verify
        # print("\n" + "="*50)
        # print("VERIFICATION - Loading CSV back into pandas:")
        # print("="*50)

        # df_loaded = pd.read_csv(output_csv)
        # print(f"Loaded DataFrame shape: {df_loaded.shape}")
        # print("DataFrame info:")
        # print(df_loaded.info())
        return df
        
    except Exception as e:
        print(f"Failed to process GRIB file: {e}")


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path

def netcdf_to_csv_london(nc_file_path, output_csv_path=None):
    """
    Read NetCDF file, extract precipitation data for London coordinates, and save as CSV.
    
    Parameters:
    nc_file_path (str): Path to the NetCDF file
    output_csv_path (str): Path for output CSV file (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with the extracted data
    """
    
    # London coordinates
    london_lat = 51.5074
    london_lon = -0.1278
    
    print(f"Reading NetCDF file: {nc_file_path}")
    
    try:
        # Open NetCDF file with xarray
        ds = xr.open_dataset(nc_file_path)
        
        print("Dataset loaded successfully!")
        print(f"Dataset dimensions: {dict(ds.dims)}")
        print(f"Dataset variables: {list(ds.data_vars)}")
        print(f"Dataset coordinates: {list(ds.coords)}")
        
        # Display dataset info
        print("\nDataset overview:")
        print(ds)
        
        # Check coordinate names (common variations)
        lat_names = ['lat', 'latitude', 'y', 'projection_y_coordinate']
        lon_names = ['lon', 'longitude', 'x', 'projection_x_coordinate']
        time_names = ['time', 'time_bnds', 'yyyymm']
        
        # Find actual coordinate names in the dataset
        lat_coord = None
        lon_coord = None
        time_coord = None
        
        for name in lat_names:
            if name in ds.coords or name in ds.dims:
                lat_coord = name
                break
                
        for name in lon_names:
            if name in ds.coords or name in ds.dims:
                lon_coord = name
                break
                
        for name in time_names:
            if name in ds.coords or name in ds.dims:
                time_coord = name
                break
        
        print(f"\nIdentified coordinates:")
        print(f"Latitude coordinate: {lat_coord}")
        print(f"Longitude coordinate: {lon_coord}")
        print(f"Time coordinate: {time_coord}")
        
        if lat_coord is None or lon_coord is None:
            print("Warning: Could not identify latitude/longitude coordinates")
            print("Available coordinates:", list(ds.coords))
            return None
        
        # Check coordinate ranges
        if lat_coord in ds.coords:
            lat_min = ds.coords[lat_coord].min().values
            lat_max = ds.coords[lat_coord].max().values
            print(f"Latitude range: {lat_min} to {lat_max}")
        
        if lon_coord in ds.coords:
            lon_min = ds.coords[lon_coord].min().values
            lon_max = ds.coords[lon_coord].max().values
            print(f"Longitude range: {lon_min} to {lon_max}")
        
        # Handle projected coordinates (if using OSGB or similar)
        if 'projection' in str(ds.coords) or lat_coord in ['y', 'projection_y_coordinate']:
            print("\nDetected projected coordinates (likely OSGB)")
            print("You may need to convert London lat/lon to the projection coordinates")
            print("For now, finding nearest grid point in projected space...")
        
        # Find nearest grid point to London
        if lat_coord in ds.coords and lon_coord in ds.coords:
            lat_idx = np.abs(ds.coords[lat_coord] - london_lat).argmin()
            lon_idx = np.abs(ds.coords[lon_coord] - london_lon).argmin()
            
            actual_lat = ds.coords[lat_coord][lat_idx].values
            actual_lon = ds.coords[lon_coord][lon_idx].values
            
            print(f"\nTarget coordinates: ({london_lat}, {london_lon})")
            print(f"Nearest grid point: ({actual_lat}, {actual_lon})")
        else:
            print("Could not find coordinate values for nearest point calculation")
            return None
        
        # Extract data for London coordinates
        selection_dict = {lat_coord: lat_idx, lon_coord: lon_idx}
        london_data = ds.isel(selection_dict)
        
        print(f"\nExtracted data variables: {list(london_data.data_vars)}")
        
        # Convert to pandas DataFrame
        df_list = []
        
        for var_name in london_data.data_vars:
            var_data = london_data[var_name]
            print(f"\nProcessing variable: {var_name}")
            print(f"Variable dimensions: {var_data.dims}")
            print(f"Variable shape: {var_data.shape}")
            
            # Get variable attributes
            if hasattr(var_data, 'attrs'):
                print(f"Variable attributes: {var_data.attrs}")
            
            # Convert to DataFrame
            if time_coord and time_coord in var_data.dims:
                # Time series data
                df_var = var_data.to_dataframe().reset_index()
                
                # Rename the variable column to be more descriptive
                if var_name in df_var.columns:
                    # Keep original variable name but add units if available
                    units = var_data.attrs.get('units', '')
                    long_name = var_data.attrs.get('long_name', var_name)
                    
                    col_name = var_name
                    if units:
                        col_name += f"_{units}".replace(" ", "_")
                    
                    df_var = df_var.rename(columns={var_name: col_name})
                    
                    # Add metadata as separate columns
                    df_var[f'{var_name}_long_name'] = long_name
                    df_var[f'{var_name}_units'] = units
                
                df_list.append(df_var)
            else:
                # Single value or no time dimension
                df_var = pd.DataFrame({
                    var_name: [var_data.values],
                    'latitude': [actual_lat],
                    'longitude': [actual_lon]
                })
                
                # Add metadata
                if hasattr(var_data, 'attrs'):
                    for attr_name, attr_value in var_data.attrs.items():
                        df_var[f'{var_name}_{attr_name}'] = str(attr_value)
                
                df_list.append(df_var)
        
        # Combine all variables into one DataFrame
        if len(df_list) > 1:
            # Find common columns for merging
            df = df_list[0]
            for df_var in df_list[1:]:
                common_cols = set(df.columns) & set(df_var.columns)
                if common_cols and len(common_cols) > 1:  # More than just the variable column
                    df = pd.merge(df, df_var, on=list(common_cols), how='outer')
                else:
                    # Concatenate side by side if same length
                    if len(df) == len(df_var):
                        df = pd.concat([df, df_var], axis=1)
                    else:
                        print(f"Warning: Cannot merge {df_var.columns} - different lengths")
        else:
            df = df_list[0] if df_list else pd.DataFrame()
        
        if df.empty:
            print("Warning: No data extracted")
            return None
        
        # Add location metadata
        df['actual_latitude'] = actual_lat
        df['actual_longitude'] = actual_lon
        df['target_latitude'] = london_lat
        df['target_longitude'] = london_lon
        df['coordinate_system'] = f"{lat_coord}, {lon_coord}"
        
        # Sort by time if time column exists
        time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
        if time_cols:
            df = df.sort_values(time_cols[0])
            print(f"Sorted by time column: {time_cols[0]}")
        
        print(f"\nFinal DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {list(df.columns)}")
        
        # Show data types
        print(f"\nData types:")
        for col in df.columns:
            print(f"  {col}: {df[col].dtype}")
        
        print(f"\nFirst few rows:")
        print(df.head())
        
        if len(df) > 5:
            print(f"\nLast few rows:")
            print(df.tail())
        
        # Save to CSV
        if output_csv_path is None:
            output_csv_path = Path(nc_file_path).stem + "_london_precipitation.csv"
        
        df.to_csv(output_csv_path, index=False)
        print(f"\nData saved to: {output_csv_path}")
        
        return df
        
    except Exception as e:
        print(f"Error processing NetCDF file: {e}")
        print("Make sure you have the required packages installed:")
        print("pip install xarray netcdf4 pandas numpy")
        raise

def main():
    # Your specific file
    nc_file = "./data/ERA5/rainfall_hadukgrid_uk_1km_ann-30y_196101-199012.nc"
    output_csv = "london_rainfall_1961_1990.csv"
    
    # Check if file exists
    if not Path(nc_file).exists():
        print(f"NetCDF file not found: {nc_file}")
        print("Please make sure the file is in the current directory or update the path.")
        return
    
    # Process the file
    try:
        df = netcdf_to_csv_london(nc_file, output_csv)
        
        if df is not None:
            # Display summary statistics
            print("\n" + "="*60)
            print("SUMMARY")
            print("="*60)
            print(f"Total records: {len(df)}")
            
            # Show time range if available
            time_cols = [col for col in df.columns if any(t in col.lower() for t in ['time', 'date', 'year'])]
            if time_cols:
                time_col = time_cols[0]
                print(f"Time range: {df[time_col].min()} to {df[time_col].max()}")
            
            # Show precipitation statistics
            precip_cols = [col for col in df.columns if any(p in col.lower() for p in ['rain', 'precip', 'precipitation'])]
            if precip_cols:
                print(f"\nPrecipitation statistics for {precip_cols[0]}:")
                print(df[precip_cols[0]].describe())
            
            # Load the CSV back into pandas to verify
            print("\n" + "="*60)
            print("VERIFICATION - Loading CSV back into pandas:")
            print("="*60)
            df_loaded = pd.read_csv(output_csv)
            print(f"Loaded DataFrame shape: {df_loaded.shape}")
            print("Sample of loaded data:")
            print(df_loaded.head(3))
            
    except Exception as e:
        print(f"Failed to process NetCDF file: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import pygrib

# Open GRIB file
grbs = pygrib.open('./data/era5_monthly_averaged_data.grib')

In [ ]:
# grib_file = './data/era5_monthly_averaged_data.grib'
# grbs2 = xr.open_dataset(grib_file, engine='cfgrib') #, backend_kwargs=backend_kwargs)

In [ ]:
# import xarray as xr

# grib_file = './data/era5_monthly_averaged_data.grib'

# # Try the first grid resolution
# try:
#     grbs2 = xr.open_dataset(grib_file, 
#                            engine='cfgrib',
#                            backend_kwargs={'filter_by_keys': {'numberOfPoints': 826}})
#     print("Successfully opened with 826 grid points")
#     print(f"Variables: {list(grbs2.data_vars)}")
#     print(f"Shape: {grbs2.dims}")
# except Exception as e:
#     print(f"Failed with 826 points: {e}")

# # Try the second grid resolution
# try:
#     grbs2_alt = xr.open_dataset(grib_file, 
#                                engine='cfgrib',
#                                backend_kwargs={'filter_by_keys': {'numberOfPoints': 207}})
#     print("Successfully opened with 207 grid points")
#     print(f"Variables: {list(grbs2_alt.data_vars)}")
#     print(f"Shape: {grbs2_alt.dims}")
# except Exception as e:
#     print(f"Failed with 207 points: {e}")

In [ ]:
# grib_file = './data/era5_monthly_averaged_data.grib'

# # Define all combinations to try
# filter_combinations = [
#     {'stepType': 'avgad', 'numberOfPoints': 826},
#     {'stepType': 'avgas', 'numberOfPoints': 826},
#     {'stepType': 'avgad', 'numberOfPoints': 207},
#     {'stepType': 'avgas', 'numberOfPoints': 207}
# ]

# datasets = {}

# for i, filters in enumerate(filter_combinations):
#     try:
#         ds = xr.open_dataset(grib_file, 
#                            engine='cfgrib',
#                            backend_kwargs={'filter_by_keys': filters})
        
#         dataset_name = f"{filters['stepType']}_{filters['numberOfPoints']}pts"
#         datasets[dataset_name] = ds
        
#         print(f"\n✅ Successfully opened: {dataset_name}")
#         print(f"   Variables: {list(ds.data_vars)}")
#         print(f"   Dimensions: {ds.dims}")
#         print(f"   Coordinates: {list(ds.coords)}")
        
#     except Exception as e:
#         print(f"❌ Failed {filters}: {str(e)[:100]}...")

# print(f"\n📊 Total datasets loaded: {len(datasets)}")

In [ ]:
# import xarray as xr
# import pandas as pd

# grib_file = './data/era5_monthly_averaged_data.grib'

# def process_all_era5_data(grib_file):
#     """Process all combinations and return organized data"""
    
#     filter_combinations = [
#         {'stepType': 'avgad', 'numberOfPoints': 826},
#         {'stepType': 'avgas', 'numberOfPoints': 826},
#         {'stepType': 'avgad', 'numberOfPoints': 207},
#         {'stepType': 'avgas', 'numberOfPoints': 207}
#     ]
    
#     all_data = {}
    
#     for filters in filter_combinations:
#         try:
#             # Open dataset
#             ds = xr.open_dataset(grib_file, 
#                                engine='cfgrib',
#                                backend_kwargs={'filter_by_keys': filters})
            
#             # Create descriptive name
#             step_type = filters['stepType']
#             grid_size = filters['numberOfPoints']
#             dataset_name = f"{step_type}_{grid_size}pts"
            
#             # Convert to DataFrame
#             df = ds.to_dataframe().reset_index()
            
#             # Add metadata
#             df['step_type'] = step_type
#             df['grid_points'] = grid_size
#             df['data_type'] = 'accumulated' if step_type == 'avgad' else 'instantaneous'
            
#             all_data[dataset_name] = {
#                 'dataset': ds,
#                 'dataframe': df,
#                 'variables': list(ds.data_vars),
#                 'shape': df.shape
#             }
            
#             print(f"✅ {dataset_name}:")
#             print(f"   Shape: {df.shape}")
#             print(f"   Variables: {list(ds.data_vars)}")
#             print(f"   Sample data types: {df.dtypes.to_dict()}")
            
#         except Exception as e:
#             print(f"❌ Failed {filters}: {e}")
    
#     return all_data

# # Process all data
# all_era5_data = process_all_era5_data(grib_file)

# # Save each dataset to CSV
# for name, data_info in all_era5_data.items():
#     df = data_info['dataframe']
#     filename = f'era5_{name}.csv'
#     df.to_csv(filename, index=False)
#     print(f"💾 Saved {name} to {filename}")

# # Display summary
# print(f"\n📋 Summary:")
# for name, data_info in all_era5_data.items():
#     print(f"{name}: {data_info['shape'][0]} rows, Variables: {data_info['variables']}")


In [ ]:
# # Open the NetCDF file
# ds = xr.open_dataset("./data/london_test_2.nc")

In [ ]:
print(ds)

In [ ]:
rain = ds['tp'].sel(latitude=51.5, longitude=0.0, method='nearest')
rain.plot()
plt.title('Daily Precipitation in London')
plt.show()

In [ ]:
# Select time series at closest grid point to London
rain_series = ds['tp'].sel(latitude=51.5, longitude=0.0, method="nearest")

In [ ]:
# Convert to pandas DataFrame
df = rain_series.to_dataframe().reset_index()
df.head()

In [ ]:
df['RR_mm'] = df['tp'] * 1000
df.head()

In [ ]:
from src.data_manipulation import transform_data_datetime

#london_data = transform_data_datetime(df=df.rename(columns={'valid_time' : 'DATE'}))

In [ ]:
# london_data.head()

In [ ]:
# Select specific variable
grb = grbs.select(name='Total precipitation')[0]
data, lats, lons = grb.data()

In [ ]:
lats

In [ ]:
lons

In [ ]:
# Read GRIB file
ds = xr.open_dataset('./data/london_data.grib', engine='cfgrib')

In [ ]:
# Try to open only GRIB2 messages
try:
    ds = xr.open_dataset('./data/london_data.grib', 
                        engine='cfgrib',
                        backend_kwargs={'filter_by_keys': {'edition': 2}})
    print("Successfully opened GRIB2 data")
except:
    # If that fails, try GRIB1
    ds = xr.open_dataset('./data/london_data.grib', 
                        engine='cfgrib',
                        backend_kwargs={'filter_by_keys': {'edition': 1}})
    print("Successfully opened GRIB1 data")

print(ds)

In [ ]:
# Explore the dataset
print("Variables:", list(ds.data_vars))
print("Coordinates:", list(ds.coords))

In [ ]:
ds['surface'].values
